## Describing Time Series Data

A time series is literally just a set of observations over time. It contains two parts, the time period, and the observed value. Most Time Series analysis techniques assume this and so in Python would typically expect a single pandas Series (not a DataFrame) with the time period as the index.

#### Autocorrelation 
is the correlation between a given point in time $Y_t$ and a previous point in time $Y_{t-n}$ . Autocovariance is related in the usual way covariance and correlation are.
The AutoCorrelation Function is the set of autocorrelations i.e. where n=1 to n=infinity.
Th fundamental premise of Time Series forecasting is predicting future values using past ones (i.e. using autocorrelation), in contrast to e.g. a more traditional Multiple Regression model which would use Independent X variables to predict $Y_t$.

The Time Series itself is sometimes referred to in the literature as the _endogenous_ (i.e. "from within"). Any additional supplementary data in the form of independent X variables are correspondingly _exogenous_ variables (i.e. "from outside").


#### Decomposition 
Most forecasting techniques form some sort of regression equation, with the prior values of $Y_t$ broken down into individual component parts. The process of breaking a time series down is called "Decomposition". Typically you would decompose an observation into:
##### $Y_t$ (observed value at time T)
##### $S_t$ (seasonal component at time T)
##### $T_t$ (trend cycle component at time T)
##### $ε_t$ (irregular or error component at time T)

Effectively, you predict $Y_t$ by extrapolating out the underlying recent trend up or down. The seasonal component is then overlaid to this to account for known and predictable fluctuations that occur seasonally. In this context seasonal can be any regular fluctuation, so it might be an xmas spike in sales over a year, but might also be a weekend spike in sales over a weekend.

#### Additive vs Multiplicative Decomposition 
There are two ways of decomposing a time series. 
We have additive decomposition if:
$Y_t = S_t + T_t + ε_t$

and we have multiplicative decomposition if:
$Y_t = S_t * T_t * ε_t$

Essentially additive decomposition assumes e.g. that the seasonal component applies some fixed amount, e.g. Saturday= "underlying trend + 1000 sales"
Multiplicative decomposition obviously multiplies the terms, e.g. Saturday = "1.2 * underlying trend".

#### Smoothing for moving averages
The first step in decomposing a time series is to estimate the trend cycle Tt. This is done using weighted moving averages. A traditional, simple, moving average can be described as a weighted-moving average whereby all the weights are the same. Weighted ones would typically weight the current observation and those nearest in time to it ($n+1$ or $n-1$) higher than those further from it (e.g. $n-5$). 
For analysis purposes it is ok to use the subsequent observations in a moving average, i.e. the window can look forwards as well as backwards to isolate the trend component.

#### Calculating seasonal component
You can subtract the Trend component, $T_t$ (i.e. the moving average part) from the overall observed $Y_t$ to be left with $ S_t + ε_t$ . The $S_t$ can then be thought of as the average of each seasonal period e.g. the average value seen in December (for multi-year trends) or the average "Monday" for weekly patterns. Importantly, you want the average of the Seasonal+Error ($ S_t + ε_t$ ), not the overall $Y_t$.
This average becomes the seasonal part.

The remaining part, error, $ε_t$ should theoretically be randomly distributed. However in real-world stuff I'm not sure how that works as e.g. you would expect Marketing to provide "Error" to the model- but structured Error.

N.B. If building a multiplicative rather than an additive model, you would divide the Trend component to get a ratio, rather than subtract.



## Forecasting Data
The above helps describe what we've seen but doesn't provide a means of forecast, or even a means of determining uplift e.g. for Campaign Eval purposes (as that uplift will be baked into the Moving Average Trend).


The simplest forecast is a moving average, whereby $Y_{t+1}$ is an average of $Y_t$ back to $Y_{t-n}$. 

#### Stationarity
Moving averages smooth over trends and seasonal differences, so are a very crude forecast. You can also only forecast one step ahead. 
Effectively they are only useful if data is "Stationary"- which means that $Y_t$ is not dependent on $t$. A stationary data set does not contain a trend or seasonal component.


### Error evaluations
There are various methods used to evaluate how good a forecast is. Some of the most prominent are:
- ME: Mean Error (average error)
- MAE: Mean Absolute Error
- MSE: Mean Squared Error (square first then take the mean of it!)
- MPE: Mean Percentage error (as a % of the observed)
- MAPE: Mean Absolute Percentage Error

Also consider...:
- Max Error

Note: assuming errors are normally distributed, which in theory they should be, then $F_{t+1}  +/-  Z* \surd (MSE)$ gives approximate confidence intervals.

$Z= 1.645$ for 90%, $Z=1.960$ for 95% and $Z=2.576$  for 99%

## Forecasting Technique 1: Single Expontential Smoothing
Where F is the forecast value:

$F_{t+1} = αY_t + (1-α) F_t$

It can only be used to predict one step ahead. Effectively it combined a weighted blen of the current observation and the forecasted value of the current observation (which in turn, was dependent on the previous value, and so on.
A high α value means that the current actual is a powerful preditor, and the "history" less so. A low α basically upweights prior forecasts, which are products of previous observations.

The model must be initialised, as obviously there is no F1. There are lots of things you can do but a good place to start is $F_2=Y_1$ (i.e. forecast for period 2= observed for period 1).

You would iterate over lots of values of α such that you minimised the mean squared error between observed and forecast values.


## Forecasting Technique 1B: Holt's Linear Exponential Smoothing
An extension of exponential smoothing to account for a local, linear trend. The trend makes it possible to forecast m time period ahead. There are two smoothing constants, α and β.

## Forecasting Technique 1C&D: Holt- Winter's Method: Additive & Multiplicative
These extend the models further to also include Seasonality, where k= the number of periods in one cycle of seasons e.g. 7 days in a week or 12 months in a year.
To initialise, we need one complete cycle of data, $s$. E.g. for weekly seasonality we need at least 7 values, obviously. 
Further to this, to initialise trend, we need $s+k$ observations. This is because for trend there needs to be some sense of change. At absolute minimum you can use $k=1$ so e.g. you would compare two mondays and assume their difference to be the underlying trend. Ideally $s=k$ so that for each seasonal variant, there is a "trend".

There are three parameters, α,β and γ

In [ ]:
ops=['add','mul'] # List the options (additive and multiplicative)
hw_combos = itertools.product([True,False],ops,ops) #Create all 8 combos of additive, multiplicative and True/False
list(hw_combos)

In [ ]:
hw_final_df,hw_selected_model_fit=Fit_Selected_HW(full_df=FTS_df,\
training_start='2019-11-10',\
forecast_start='2020-02-29 19:00:00',\
forecast_end='2020-03-05 12:00:00',\
var_to_forecast='N',periods_seasonality=168)

hw_selected_model_fit.summary()

In [ ]:
hw_forecast = hw_selected_model_fit.predict(start='2020-02-29 19:00:00', end='2020-03-05 12:00:00')
hw_actuals= FTS_df['2020-02-29 19:00:00':'2020-03-05 12:00:00']


actual_vs_forecast=pd.DataFrame(hw_final_df['modelled'].append(hw_forecast)).join(pd.DataFrame(FTS_df['N']))
actual_vs_forecast['2020-02-21':].plot()

In [ ]:
actual_vs_forecast['2020-02-28':]

In [ ]:
hw_uplift=pd.DataFrame(hw_forecast).join(hw_actuals).rename(columns={0:'expected'})
hw_uplift['uplift']=hw_uplift['N']-hw_uplift['expected']
hw_uplift

In [ ]:
hw_uplift.uplift.sum()

In [ ]:
hw_uplift['2020-03-05':'2020-03-06'].uplift.sum()

In [ ]:
actual_vs_forecast[['actual','expected']]['2020-02-28':'2020-03-05'].plot()

# Forecasting Technique 2: ARIMA
An alternative and slightly more complex (note, not always better!) technique is a class of methods broadly called **ARIMA**.
These are based on the Autocorrelation Function (ACF) and is more refined than a moving averages method.
To build an ARIMA, there are a number of steps to take, in addition to those used in e.g. a Moving Averages method.

Within the class of **ARIMA** there are three major subdivisions:
* **ARIMA**: The basic class that uses prior _n_ observations in the Time Series to make a forecast  
* **S**ARIMA: An expansion of ARIMA that adds the "S" for "Seasonal". This additionally uses _n*s_ observations. For example if for daily data there was a 7 day weekly recurrent pattern, _s=7_ and so it would use observations 7, 14, 21 days ago for _n=1,2,3_ respectively   
* SARIMA**X** : A further expansion of SARIMA. The "X" stands for "exogenous" variables. It is effectively a means of blending a more typical regression equation to further boost the model's effectiveness. For example, you might see that sales are broadly trending up, and have a weekly seasonal pattern, but marketing spend might completely change the status quo. If marketing spend were included as an exogenous variable, you could get greatly improved accuracy. 

## ARIMA Step 1: Autocorrelation
Auto correlation is the correlation between an observation and another observation in the same series at a different point in time, e.g. between $Y_t$ and $Y_{t-7}$.

The Autocorrelation Function (ACF) is the full distribution of correlation coefficients up to $t-N$ moments in time ago.
However the ACF doesn't tell the full story. Many $Y_t$ values will be highly correlated with $Y_{t-1}$, for example. And so that means that because $Y_{t-1}$ is highly correlated with $Y_{t-2}$, there will also be a decent agreement between $Y_t$ and $Y_{t-2}$. This can mask true repeating correlations. 

Therefore we have the:
#### Partial Autocorrelation Function (PACF).

This produces an ACF that removes the effect of other time lags and produces something independent. This is called "partialling out". The PACF is calculated using an AutoRegression (AR), which is literally a regression on the previous Yt-n values. PACF is derived from the coefficients of that regression model.

In [ ]:
# Create a new clean dataset to test Arima
ARIMA_training=pd.DataFrame(training['N']) #Must specify the "pd.DataFrame" else it'll return a Series which has different properties

# Correlation with itself (yesterday)
print("Correlation at lag 1 is :",ARIMA_training['N'].autocorr())

# Correlation with itself (7 days ago)
print("Correlation at lag 24 is :",ARIMA_training['N'].autocorr(24))

# Correlation with exactly same record (useless sanity check)
print("Correlation at lag 168 is :",ARIMA_training['N'].autocorr(168))

# Plot autoregression to see how it correlates with each lag
autocorrelation_plot(ARIMA_training['N'])
pyplot.show()

# Get Partial ACF
from statsmodels.tsa.stattools import pacf

PACF_values=pd.DataFrame(pacf(ARIMA_training['N'],nlags=200)) #PACF numpy array converted to df
PACF_values.plot() #Plot
pyplot.show()

# In this example it appears a PACF can be >1. We have some erratic values going back beyond 25 instances which mask the pattern
sub_df=PACF_values[0:1000]
sub_df.plot(style='.-',xlim=(0,26))
pyplot.show()

PACF_values

## ARIMA Step 2: White Noise
A forecasting model is deemed 'well suited' if the forecast errors are purely random. If this is the case, the residuals are described as "white noise".
If a time series is white noise, both AC and PAC coefficients are approximately independent and normally distributed with mean 0 and standard deviation of $1/\surd n$.
Hence it is useful to plot ACF and PACF with a range of $+/- 1.96/\surd(n)$. Coefficients outside these boundaries are probably not white noise i.e. significant.


In [ ]:
import math
1.96*math.sqrt(25)

## ARIMA Step 3: Recognising non-stationarity
First you have to examine the time plot. If the mean or variance change with time it isn't stationary.
ACF and PACF also give evidence, as autocorrelations of stationary data drop to zero quite quickly, whilst those for non-stationary data can take a number of lags to do so. The PACF of a non-stationary time series will typically have a large coefficient close to 1 at lag 1.

## ARIMA Step 4: Removing non-stationarity- by Differencing
It is important to remove trends from time series data prior to model building since such autocorrelations dominate the ACF. This can be done by differencing. 
A differenced series is the change in each observation in a time series e.g. $Y_t'= Y_t - Y_{t-1}$



Occasionally, removing such differences still doesn't result in non-stationarity, i.e. there is a trend in the changes. Ordinarily, a second round of differencing does the trick i.e. $Y_t''= Y_t'- Y_{t-1}'$
Basically you difference the already-differenced data.

When the differenced data is effectively white noise you know you've done it enough.


### Seasonal Differencing

Seasonal Differencing is the same process but rather than taking Yt- Yt-1, you do Yt- Yt-n where n= the number of periods after which seasonality plays a part, e.g. 7 days in a week, 12 months in a year. It can be repeated to obtain second order seasonal differencing but this is rarely needed.

## ARIMA Step 5: Unpicking an ARIMA Model

ARIMA stands for AR - I - MA
or 
Autoregression - I (differences) - Moving Average

#### AR
An AR model basically predicts $Y_t$ using a regression of  
$ β_1 Y_{t-1} + β_2 Y_{t-2} + ... β_p Y_{t-p} + c + ε_{t} $

It takes one parameter ($p$)- which is the number of time lags you use to make a the prediction.

There are constraints on the coefficients:
- If $p=1$ then $-1 < β_1 < 1$
- If $p=2$ then $-1 < β_2 < 1$  AND $β_1+β2<1$ AND $β_2-β_1<1$
- If $p=3$ even more conditions hold

#### I
The "I" in ARIMA details the differencing applied. So if first order differencing is applied, then the parameter($d$)=1. If second order differencing is necessary then the parameter,$d$=2.

#### MA
An MA model, confusingly has nothing to do with Moving Averages in the traditional sense. Instead it predicts $Y_t$ but regressing on the historical errors. The equation looks identical to above in AR except that $Y_{t-k}$ is replaced with $ε_{t-k}$

The parameter dictacting how many time lags to use is called q.
The coefficients are again constrained.
An MA(1) and an AR(1) model are mirror images.

In truth, I'm not sure that an MA model in isolation would do much. However I think the value comes in conjunction with the AR-I parts, as it would help to ensure no systematic error or deviance over time.

#### Combining for an ARIMA model $(p,d,q)$
Combining the above creates a model whereby we have three parameters $(p,d,q)$:
- $p$ is how many times to autoregress, i.e. how many lags on observed values $Y_{t-k}$ to use to build the model. Rarely above 2. A slow decline in ACF but fast decline in PACF characterises an AR(1) model
- $d$ is how many time to difference the data to achieve stationarity, again rarely above 2
- $q$ is how many lags to regress on the error term. An ACF with a single peak at lag 1, and negative PACF values dying down to 0 characterise and MA(1) model.

#### SARIMA $(p,d,q) (P,D,Q)s$ models
Standard $(p,d,q)$ models can also incorporate a seasonal component. Where the standard $(p,d,q)$ model is regressing on recent observations to predict the next, i.e. yesterday and maybe the day before, adding $(P,D,Q)s$ adds the seasonal component e.g. last Saturday. 

So for example I can predict tomorrow (Weds) by looking at today (Tuesday) and last Wednesday because of seasonality. 
$(P,D,Q)$ perform the same function as $(p,d,q)$ but $s$ periods ago. So $p=1$ would autoregress on yesterday, $P=2$ and $s=7$ would autoregress on 7 and 14 periods ago. 


## ARIMA: Step 6: Explaining it and writing it

Convention is that ARIMA models are written using the "backshift" operator, $B$ which shifts data back one period, i.e.
$BY_t =Y_{t-1}$


Two applications of $B$ shift the data back two period. i.e. $B(BY_t) = B^2Y_t = Y_{t-2}$

For monthly data, we can use the notation $B^12$ to shift back to the same month last year:
$B^12Y_t = Y_{t-12}$

Therefore in general, $B^s$ represents "shift back $s$ time periods".

A first order *difference* is represented by $(1-B)$

$Y_t' = Y_t - Y_{t-1} $ and so $= Y_t - BY_t = (1-B)Y_t$

Similarly, a second order difference is represented as $(1-B)^2$

A backshift notation is convenient because terms can be multiplied together to see the combined effect. For example,  a first difference followed by a first seasonal difference can be written as:
$$(1-B)(1-B^S)Y_t = (1-B-B^S +B^{S+1})Y_t =Y_t -Y_{t-1} -Y_{t-S} +Y_{t-S+1}$$

#### An AR model can be written as 

$$(1-\varphi_1B - ... - \varphi_pB^p)Y_t = c + \epsilon_t$$

(where $\varphi$ is the regression coefficient on the previous time lags $B$)


#### An MA model can be written as 

$$ Y_t = c + (1-\theta_1B - ... - \theta_qB^q)\epsilon_t$$

(where $\theta$ is the regression coefficient on the previous error terms $\epsilon$)

#### An ARIMA $(p,d,q)$ model can be written as:

$$(1-\varphi_1B - ... - \varphi_pB^p)(1-B)^dY_t=(1-\theta_1B - ... - \theta_qB^q)\epsilon_t$$

#### An ARIMA $(p,d,q)(P,D,Q)s$ model can be written as:

$$(1-\varphi_1B - ... - \varphi_pB^p)(1-\Phi_1B - ... - \Phi_PB^SP)(1-B)^d(1-B^S)^DY_t$$
$$=(1-\theta_1B - ... - \theta_qB^q)(1-\Theta_1B^S - ... - \Theta_QB^sQ)\epsilon_t$$

As an example, a $(0,1,1)(0,1,1)12$ model can be written as:
$$(1-B)(1-B^{12})Y_t=(1-\theta_1B)(1-\Theta_1B^{12})\epsilon_t$$

# ARIMA Step 7: Measuring it
In addition to the backshift notation, $\sigma$ denotes the standard deviation of the errors, $\epsilon_t$

All the parameters $(\varphi, \theta, \Theta, \Phi) $ etc. have to be estimated. This is done using *ordinary least squares (OLS)* or *maximum likelihood estimation (MLE)*

#### Portmanteau tests for white noise
Insstead of examining each individual autocorrelation coefficient, 'portmanteau' tests can be used which consider the coefficients taken together. An example is the Ljung-Box text which employs the $Q$ statistic:
$$ Q^*=n(n+2)\sum_{k=1}^h(n-k)^{-1}r_k^2$$

where $n$ is the number of observations,
$h$ is the maximum time lag considered, and $r_k$ is the autocorrelation at lag $k$. 

If the residuals of the model are white noise (which you want) $Q^*$ has chi-square $\chi^2$ distribution with $(h-m)$ degrees of freedom, where $m$ is the number of parameters in the model.

If the $Q^*$ statistic lies in the right hand 5% tail of the $\chi^2$ distribution (i.e. P<0.05) it is normally concluded that the data are not white noise.
So you want it to be >0.05 but equally as this is a low bar, it isn't sufficient as a test of model quality.

#### AIC for model comparison
Several models may be a good fit, so we need to distinguish between them. Indeed the more parameters you use, the better your fit will inevitably be, but this could just be overfitting of randomness. AIC is useful because it penalises overfitting in measuring model fit.

$$ AIC = -2LogL + 2m $$
where $L$ is the likelihood and $m= p +q +P +Q$

The **lower** the AIC score the better.


### Let's get modelling ARIMA!